In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
score_map: Dict[str, float] = {}
df = pd.read_csv("input/ruddit/Ruddit.csv", engine="c", low_memory=False)
for t in df.itertuples():
    k = getattr(t, "post_id") + "_" + getattr(t, "comment_id")
    score_map[k] = getattr(t, "offensiveness_score")

In [4]:
%%time
df = pd.read_csv("input/ruddit/ruddit_with_text.csv", engine="c", low_memory=False)
blacklist = {"[deleted]", "[removed]"}
rows = []
for t in df.itertuples():
    text = getattr(t, "txt")
    s = text.strip().lower()
    if len(s)==0 or s in blacklist:
        continue
    k = getattr(t, "post_id") + "_" + getattr(t, "comment_id")
    rows.append({"bws": score_map[k], "text": text})
df = pd.DataFrame.from_records(rows)
df["bws"] = df["bws"].astype(np.float32) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   bws     5710 non-null   float32
 1   text    5710 non-null   object 
dtypes: float32(1), object(1)
memory usage: 67.0+ KB
Wall time: 63 ms


In [5]:
scml.find_missing_values(df)

,Total,Percent,Type
bws,0,0.0,float32
text,0,0.0,object


# Preprocess Text
Speed dropped from 2400 to 600 it/s

In [7]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


def pre3(row) -> str:
    return mylib.pre3(row["text2"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)
col = "text3"
print(col)
df[col] = df.progress_apply(pre3, axis=1)

text1


100%|██████████████████████████████████████████| 5710/5710 [00:10<00:00, 555.64it/s]


text2


100%|███████████████████████████████████████████| 5710/5710 [04:06<00:00, 23.13it/s]


text3


100%|██████████████████████████████████████████| 5710/5710 [00:53<00:00, 107.23it/s]


In [8]:
cols = ["bws", "text", "text1", "text2", "text3"]
df[cols].sample(20).sort_values("bws").head(20)

,bws,text,text1,text2,text3
3326,-0.604,I see you too are a man of culture;),I see you too are a man of culture [Wink or smirk],I see you too are a man of culture [Wink or smirk],i see you too be a man of culture wink or smirk
693,-0.522,"Not completely open, but a lot more open than they are right now.","Not completely open, but a lot more open than they are right now.","Not completely open, but a lot more open than they are right now.",not completely open but a lot more open than they be right now
4898,-0.521,"Wait, so how did you get here then?","Wait, so how did you get here then?","Wait, so how did you get here then?",wait so how do you get here then
1763,-0.417,Took a call from her ex.,Took a call from her ex.,Took a call from her ex.,take a call from her ex
4745,-0.326,"this one is a beast. Runs for three days straight, then you just charge it for about one hour at lunch time and it's 100% again, ready for another three days. Absolutely love it.\n\nNot that great of a camera though, but I couldn't care less","this one is a beast. Runs for three days straight, then you just charge it for about one hour at lunch time and it's 100% again, ready for another three days. Absolutely love it. Not that great of a camera though, but I couldn't care less","this one is a beast. Runs for three days straight, then you just charge it for about one hour at lunch time and it is 100% again, ready for another three days. Absolutely love it. Not that great of a camera though, but I could not care less",this one be a beast runs for three day straight then you just charge it for about one hour at lunch time and it be 100 again ready for another three day absolutely love it not that great of a camera though but i could not care less
5280,-0.292,I bumped into Martin O'Malley once. At least I think I did.,I bumped into Martin O'Malley once. At least I think I did.,I bumped into Martin O'Malley once. At least I think I did.,i bump into martin o malley once at least i think i do
3222,-0.208,"Additionally, adjacent can mean on the flat side of a square or a corner. The highest number that could be seen is ""8"", that would mean that that square is completely surrounded by mines.\n\nEdit: It is the most basic deduction game, but, in my experience, there is a ton of luck involved too.","Additionally, adjacent can mean on the flat side of a square or a corner. The highest number that could be seen is ""8,"" that would mean that that square is completely surrounded by mines. Edit: It is the most basic deduction game, but, in my experience, there is a ton of luck involved too.","Additionally, adjacent can mean on the flat side of a square or a corner. The highest number that could be seen is ""8,"" that would mean that square is completely surrounded by mines. Edit: It is the most basic deduction game, but, in my experience, there is a ton of luck involved too.",additionally adjacent can mean on the flat side of a square or a corner the high number that could be see be 8 that would mean that square be completely surround by mine edit it be the most basic deduction game but in my experience there be a ton of luck involve too
5297,-0.167,The exact reason she should never get it.,The exact reason she should never get it.,The exact reason she should never get it.,the exact reason she should never get it
4974,-0.104,"It seems highly illogical to believe a. Water retains a “memory” of its previous solvent, and b. That some how the smaller the amount of the solvent the greater the potency, which inversely exponentially increases.","It seems highly illogical to believe a. Water retains a ""memory"" of its previous solvent, and b. That some how the smaller the amount of the solvent the greater the potency, which inversely exponentially increases.","It seems highly illogical to believe a. Water retains a ""memory"" of its previous solvent, and b. That some how the smaller the amount of the solvent the greater the potency, which inversely exponentially increases.

In [9]:
df[cols].sample(20).sort_values("bws").head(20)

,bws,text,text1,text2,text3
2636,-0.340,Xiaomi Hybrid has a nice and good sounding v-shape to it. I like mine a lot for electronic music and anything that comes with a punchy bass.\n\nHaven't heard anything good about the Youth edition.,Xiaomi Hybrid has a nice and good sounding v-shape to it. I like mine a lot for electronic music and anything that comes with a punchy bass. Haven't heard anything good about the Youth edition.,"Xiaomi Hybrid has a nice and good sounding v-shape to it. I like mine a lot for electronic music and anything that comes with a [punchy; having punch; forceful, spirited] bass. have not heard anything good about the Youth edition.",xiaomi hybrid have a nice and good sounding v shape to it i like mine a lot for electronic music and anything that come with a punchy having punch forceful spirited bass have not hear anything good about the youth edition
3844,-0.333,"Yours is the Girl Scout Pledge. What u/hagnonbg wrote was the Girl Scout Law. 2 different things, both correct.\n\nand the something something is ""to help people at all times""","Yours is the Girl Scout Pledge. What u/hagnonbg wrote was the Girl Scout Law. 2 different things, both correct. and the something something is ""to help people at all times""","Yours is the Girl Scout Pledge. What u/hagnonbg wrote was the Girl Scout Law. 2 different things, both correct. and the something is ""to help people at all times""",yours be the girl scout pledge what u hagnonbg write be the girl scout law 2 different thing both correct and the something be to help people at all time
1096,-0.312,"Humans do inherit traits, but they do NOT inherit learned behaviours. They may mimic, but that’s entirely different and can be unlearned as well as it was learned.","Humans do inherit traits, but they do NOT inherit learned behaviours. They may mimic, but that's entirely different and can be unlearned as well as it was learned.","Humans do inherit traits, but they do NOT inherit learned behaviours. They may mimic, but that is entirely different and can be unlearned as well as it was learned.",human do inherit trait but they do not inherit learn behaviour they may mimic but that be entirely different and can be unlearn as well as it be learn
3457,-0.191,"You said you recently installed geopandas. Will you share how did you install it?\n\nI tried conda install -n py36 -c conda-forge install geopandas.\n\nIt installs good, but when i import geopandas it gives ImportError relating to fiona.\n\n dlopen some .so file, ... referenced from, ... library not found.\n\n\nDid you get any library linking error, if so how did you fixed it?\n\nMy computer:\nMacos mojave\nAnaconda \nPy36 is anaconda environment i created.\n\nThanks.","You said you recently installed geopandas. Will you share how did you install it? I tried conda install -n py36 -c conda-forge install geopandas. It installs good, but when i import geopandas it gives ImportError relating to fiona. dlopen some .so file, ... referenced from, ... library not found. Did you get any library linking error, if so how did you fixed it? My computer: Macos mojave Anaconda Py36 is anaconda environment i created. Thanks.","You said you recently installed geopandas. Will you share how did you install it? I tried conda install -n py36 -c conda-forge install geopandas. It installs good, but when i import geopandas it gives ImportError relating to fiona. dlopen some .so file, ... referenced from, ... library not found. Did you get any library linking error, if so how did you fixed it? My computer: Macos mojave Anaconda Py36 is anaconda environment i created. Thanks.",you say you recently instal geopandas will you share how do you install it i try conda install n py36 c conda forge install geopandas it install good but when i import geopandas it give importerror relate to fiona dlopen some so file reference from library not find do you get any library link error if so how do you fix it my computer macos mojave anaconda py36 be anaconda 

In [10]:
df[cols].sample(20).sort_values("bws").head(20)

,bws,text,text1,text2,text3
4842,-0.457,Exposing the number of bots would crash their stock,Exposing the number of bots would crash their stock,Exposing the number of bots would crash their stock,expose the number of bot would crash their stock
121,-0.438,"Still not 2.5 inches. 1 inch is about as long as their canine teeth get, and the older they are, the shorter the teeth.","Still not 2.5 inches. 1 inch is about as long as their canine teeth get, and the older they are, the shorter the teeth.","Still not 2.5 inches. 1 inch is about as long as their canine teeth get, and the older they are, the shorter the teeth.",still not 2 5 inch 1 inch be about as long as their canine tooth get and the old they be the short the tooth
4204,-0.396,For anyone that didn’t bother actually reading the article this is a satire bill.,For anyone that didn't bother actually reading the article this is a satire bill.,For anyone that did not bother actually reading the article this a satire bill.,for anyone that do not bother actually read the article this a satire bill
1220,-0.340,I agree. Some locations just have a ton of relatively healthy food to eat for cheap. Some places are literally food deserts where you need to travel a bit to even buy produce. Theres a big gradient between them that make it harder or easier to eat healthy.,I agree. Some locations just have a ton of relatively healthy food to eat for cheap. Some places are literally food deserts where you need to travel a bit to even buy produce. Theres a big gradient between them that make it harder or easier to eat healthy.,I agree. Some locations just have a ton of relatively healthy food to eat for cheap. Some places are literally food deserts where you need to travel a bit to even buy produce. Theres a big gradient between them that make it harder or easier to eat healthy.,i agree some location just have a ton of relatively healthy food to eat for cheap some place be literally food desert where you need to travel a bit to even buy produce there s a big gradient between they that make it hard or easy to eat healthy
1112,-0.333,"Ok, then I would argue I do not believe any current evidence of the contrary to my assertion.\n\nThere is no evidence that I am aware of that tells me otherwise.","Ok, then I would argue I do not believe any current evidence of the contrary to my assertion. There is no evidence that I am aware of that tells me otherwise.","Ok, then I would argue I do not believe any current evidence of the contrary to my assertion. There is no evidence that I am aware of that tells me otherwise.",ok then i would argue i do not believe any current evidence of the contrary to my assertion there be no evidence that i be aware of that tell i otherwise
148,-0.255,But you're not weakening them with the ban. You are strengthening them by validating their persecution narrative. If your house is on fire you have a choice. Use a water to douse the fire or try to protect yourself by throwing logs at the flames. We don't get safer by adding fuel to the fire. \n\n,But you're not weakening them with the ban. You are strengthening them by validating their persecution narrative. If your house is on fire you have a choice. Use a water to douse the fire or try to protect yourself by throwing logs at the flames. We don't get safer by adding fuel to the fire.,But you are not weakening them with the ban. You are strengthening them by validating their persecution narrative. If your house is on fire you have a choice. Use a water to douse the fire or try to protect yourself by throwing logs at the flames. We do not get safer by adding fuel to the fire.,but you be not weaken they with the ban you be strengthen they by validate their persecution narrative if your house be on fire you have a choice use a water to douse the fire or try to protect yourself by throw log at the flame we do not get safe by add fuel to the fire
366,-0.229,"To be honest, fostering national or international amateur sports competit

# Review data

In [11]:
col = "worker"
df[col] = 0
df[col] = df[col].astype(np.int8)
cols = ["bws", "worker", "text", "text1", "text2", "text3"]
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   5710 non-null   int32  
 1   bws     5710 non-null   float32
 2   worker  5710 non-null   int8   
 3   text    5710 non-null   object 
 4   text1   5710 non-null   object 
 5   text2   5710 non-null   object 
 6   text3   5710 non-null   object 
dtypes: float32(1), int32(1), int8(1), object(4)
memory usage: 228.7+ KB


In [12]:
%%time
df[cols].to_parquet("output/pre_ruddit.parquet", index=False)

Wall time: 53 ms
